# Federated learning - Client aggregation

### Imports

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import copy

### Prepare the data

##### MNIST

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_data, train_data_labels, test_data, test_data_labels = [[[] for i in range(10)] for i in range(4)]
    
list(map(lambda a,b:sort_lists(train_data,train_data_labels, a, b), train_images, train_labels))
list(map(lambda a,b:sort_lists(test_data,test_data_labels, a, b), test_images, test_labels))

train_data, train_data_labels, test_data_labels, test_data_labels = np.asarray(train_data), np.asarray(train_data_labels), np.asarray(test_data_labels), np.asarray(test_data_labels)
print(len(train_data[0]), len(train_data_labels[0]), len(test_data[0]), len(test_data_labels[0]))

5923 5923 980 980


a:\projects\python sesong 2\env\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


###### CIFAR10

In [ ]:
(cifar_images, cifar_labels), (cifar_test_images, cifar_test_labels) = datasets.cifar10.load_data()
cifar_train_data, cifar_train_labels, cifar_test_data, cifar_test_labels = [[[] for i in range(10)] for i in range(4)]


### Models

###### MNIST

In [ ]:
mnist_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])

mnist_model.compile(
                optimizer=tf.keras.optimizers.SGD(0.01),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

###### CIFAR10

In [ ]:
# TODO

### Classes and methods

###### Server class

In [4]:
class Server():
    def __init__(self, model):
        self.clients = []
        self.client_weights = []
        self.weights = []
        self.model = model
        self.model.compile(
                optimizer=tf.keras.optimizers.SGD(0.1),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )
        
    # Implementing a getter because TF get_weights returns a list rather than array.
    def get_weights(self):
        return np.asarray(self.model.get_weights())
    
    # Performs a single round of training for set amount of epochs    
    def train_clients(self):
        for c in self.clients:
            self.client_weights.append(c.train())
        return self.client_weights
    
    # Updates weights for server and clients
    def update_weights(self, new_weights):
        self.weights = new_weights
        for c in self.clients:
            client.update_weights(self.get_weights())
            
    # Performs federated averaging for given amount of rounds
    def federated_averaging(self, rnds):
        average_weights = np.asarray(self.model.get_weights())
        # Training for a specified amount of rounds
        for rnd in range(rnds):
            t_weights = []
            total_samples = 0
            # Training loop - trains each client for a single round for a specified amount of epochs
            for client in self.clients:
                t_weights.append(client.train())

                total_samples += client.get_sample_amount()

            tmp_weights = [t_weights[i] * len(self.clients[i].dataset.testing_data) for i in range(len(self.clients))]
            tmp_weights = [w / total_samples for w in average_weights]

            average_weights = [x + y for x, y in zip(init_weights, tmp_weights)]

        self.update_weights(average_weights)
        print('FINISHED')

###### Client class

In [5]:
# Class representing the client and the necessary methods a client requires.
class Client():
    def __init__(self, model, dataset):
        self.model = model
        self.dataset = dataset
        self.active_weights = self.get_weights()
        self.rep_weights = self.active_weights
        
    # Implementing a getter because TF get_weights returns a list rather than array.
    def get_weights(self):
        return np.asarray(self.model.get_weights())
    
    # Getter just for better readability
    def get_sample_amount(self):
        return len(self.dataset.training_data)
    
    def train(self, epochs):
        self.model.fit(
                self.dataset.training_data, 
                self.dataset.training_labels, 
                epochs=LOCAL_EPOCHS, 
                validation_data=(
                    self.dataset.testing_data,
                    self.dataset.testing_labels
                ))
        
        self.rep_weights = np.asarray(self.model.get_weights())
        return self.rep_weights
    
    def update_weights(self, new_weights):
        assert self.active_weights.shape == new_weights.shape, "Shapes for the input weights are not the same"
        self.active_weights = np.mean( np.array([ self.active_weights, new_weights ]), axis=0 )
        self.model.set_weights(self.active_weights)
        
        
    

###### Dataset class

In [16]:
class Dataset():
    def __init__(self, training_data, training_labels, testing_data, testing_labels):
        self.training_data = np.asarray(training_data)
        self.training_labels = np.asarray(training_labels)
        self.testing_data = np.asarray(testing_data)
        self.testing_labels = np.asarray(testing_labels)

###### Utility methods

In [ ]:
def set_client_weights(weights, clients):
    for c in clients:
        c.model.set_weights(weights)

In [2]:
def sort_lists(list_a, list_b, data_entry, data_label):
    list_a[data_label].append(data_entry)
    list_b[data_label].append(data_label)

### Initialize variables

##### Constants

In [ ]:
NUMBER_OF_CLIENTS = 10 # Currently only 10 data splits
LOCAL_EPOCHS = 3
ROUNDS = 1

##### Objects

In [13]:
server = Server(mnist_model)
init_weights = np.asarray(server.model.get_weights())

# Populate the server's client list with clients holding data
server.clients = [Client(copy.deepcopy(mnist_model), Dataset(train_data[i], train_data_labels[i], test_data[i], test_data_labels[i])) for i in range(NUMBER_OF_CLIENTS)]

##### Execution

In [ ]:
# Initialize the clients' weights
set_client_weights(init_weights, server.clients)

# Perform federated averaging on the clients
server.federated_averaging(ROUNDS)

# Testing - delete later

In [15]:

#model.set_weights(init_weights)
def FedAvg():
    weights = []
    average_weights = np.asarray(model.get_weights())
    # Training for a specified amount of rounds
    for rnd in range(ROUNDS):
        t_weights = []
        total_samples = 0
        # Training loop - trains each client for a single round for a specified amount of epochs
        for client in clients:
            t_weights.append(client.train())
            
            total_samples += client.get_sample_amount()
            
        tmp_weights = [t_weights[i] * len(clients[i].dataset.testing_data) for i in range(NUMBER_OF_CLIENTS)]
        tmp_weights = [w / total_samples for w in average_weights]
        
        average_weights = [x + y for x, y in zip(init_weights, tmp_weights)]
        
    model.set_weights(average_weights)
    print('FINISHED')
    


Epoch 1/3
186/186 [==============================] - 11s 6ms/step - loss: 0.2625 - sparse_categorical_accuracy: 0.9946 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/3
186/186 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/3
186/186 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 1/3
211/211 [==============================] - 1s 5ms/step - loss: 0.1612 - sparse_categorical_accuracy: 0.9953 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/3
211/211 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/3
211/211 [==============================] - 1s 4ms/step - loss:

AttributeError: 'list' object has no attribute 'update_weights'

In [19]:
server.model.get_weights()

[array([[ 0.06603812,  0.06138564,  0.01882748, ..., -0.0106024 ,
         -0.04984839, -0.07946195],
        [-0.05135015,  0.02872749, -0.05567926, ..., -0.02771584,
          0.00838172,  0.03980216],
        [ 0.04505598, -0.07869708, -0.04524029, ...,  0.04245869,
         -0.03200928, -0.06609197],
        ...,
        [ 0.04489557,  0.06759686,  0.06125114, ...,  0.02124279,
          0.03095571,  0.04465731],
        [ 0.02795984,  0.03932033, -0.00414574, ...,  0.07700751,
          0.05062222, -0.04201375],
        [-0.04542928,  0.04772121, -0.01172901, ...,  0.02784236,
         -0.07734098,  0.05741563]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
def FedCD:

In [68]:
history = model.fit(
    np.asarray(clients[0].dataset.training_data), 
    np.asarray(clients[0].dataset.training_labels), 
    epochs=10, 
    validation_data=(
        np.asarray(clients[0].dataset.testing_data), 
        np.asarray(clients[0].dataset.testing_labels)
    )
)

Epoch 1/10
186/186 [==============================] - 1s 3ms/step - loss: 0.9521 - sparse_categorical_accuracy: 0.9936 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/10
186/186 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/10
186/186 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/10
186/186 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/10
186/186 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000
Epoch 6/10
186/186 [==============================] - 1s 3ms/ste

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'], label='accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(np.asarray(orderedListDataTest[0]),  np.asarray(orderedListLabelsTest[0]), verbose=2)


In [ ]:
NUMBER_OF_MODELS = 10
models = []
for i in range(NUMBER_OF_MODELS):
    
    models.append(
        Client(model.fit(
            np.asarray(orderedListData[i]), 
            np.asarray(orderedListLabels[i]), 
            epochs=10, 
            validation_data=(
                np.asarray(orderedListDataTest[i]), np.asarray(orderedListLabelsTest[i])
            )), len(orderedListData[i])))

In [ ]:
(models[0].model.get_weights() == models[1].model.get_weights()).all()

In [ ]:
np.array_equal(models[0].model.get_weights(), models[1].model.get_weights())

In [ ]:
models[0].sample_amount

In [ ]:
train_data = [[] for i in range(10)]
train_data

In [ ]:
[train_data[train_labels[i]].append(train_images[i]) for i in range(len(train_images))]

In [ ]:
len(train_data[9])

In [ ]:
train_labels[0]

In [42]:
test_weights = np.asarray(init_weights) + 1

In [45]:
(test_weights + np.asarray(init_weights)) / 2

a:\projects\python sesong 2\env\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([array([[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]], dtype=float32),
       array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5

In [34]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

In [35]:
np.asarray(model.get_weights())

array([array([[ 0.02979872, -0.07496471,  0.06127212, ...,  0.00026497,
         0.01988209,  0.02875005],
       [-0.06187219, -0.07302045, -0.06389229, ..., -0.04385625,
        -0.00143486,  0.0573824 ],
       [-0.06367228, -0.03734235, -0.01923789, ..., -0.04568401,
        -0.07224934, -0.05466868],
       ...,
       [ 0.06434707, -0.04622708, -0.01274758, ...,  0.0193691 ,
         0.08004429,  0.04332334],
       [ 0.0633579 ,  0.03055925, -0.06869059, ..., -0.06149226,
         0.0326076 ,  0.06383979],
       [-0.037656  , -0.00455572, -0.0554471 , ...,  0.03669924,
        -0.02400378, -0.00825565]], dtype=float32),
       array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [29]:
test_data[0][0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  11,
        150, 253, 202,  31,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  37,
        251, 251, 253, 107,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  